In [1]:
import os
from pathlib import Path
from omegaconf import OmegaConf
from hydra.utils import instantiate

from emg2qwerty import transforms, utils
from hydra import initialize, compose
from emg2qwerty.data import WindowedEMGDataset
from torch.utils.data import DataLoader, Dataset

from emg2qwerty.transforms import Compose, ToTensor, LogSpectrogram, TemporalAlignmentJitter, IdentityTransform, RandomBandRotation, SlidingCovariance, SpecAugment, ZNormalizeTime

import pytorch_lightning as pl
import logging
from emg2qwerty.lightning import WindowedEMGDataModule
import numpy as np

import torch
import pickle

In [ ]:
""" Train the emg2qwerty model. 

Code taken from: 
Sivakumar, Viswanath, et al. 
"emg2qwerty: A large dataset with baselines for touch typing using surface electromyography." 
Advances in Neural Information Processing Systems 37 (2024): 91373-91389.

https://github.com/facebookresearch/emg2qwerty
"""

In [ ]:
""" 
According to your setup, you need to change,

dataset:
  root: /mnt/dataDrive/qwertyData/data in base.yaml

and 
lm_path: /mnt/dataDrive/emgFullCorpora/toUpload/emg2qwerty/models/wikitext-103-6gram-charlm.bin in ctc_beam.yaml

in config files.
"""


"""
All codes are same except that we use SlidingCovariance instead of log spectrogams.
We also train for 250 epochs instead of 150 epochs since our model took longer to converge. 

The rest of the things are same as in https://github.com/facebookresearch/emg2qwerty.
This is a controlled experiment to show that covariance features rae better than spectral features. 
"""

In [2]:
"""deocder = ctc_beam or ctc_greedy
approxDiag = (keep TRUE)
"""

user = "user0"
decoder = "ctc_greedy" 
approxDiag = True

In [3]:
with initialize(config_path = "config", version_base = None):
    config = compose(
        config_name = "base",
        overrides=[
            f"user={user}",
            f"decoder={decoder}"
        ]
    )

In [4]:
with open("/mnt/dataDrive/emgFullCorpora/toUpload/DATA/emg2qwerty/frechetMean/" + user + "Mean.pkl", "rb") as f:
    userMean = pickle.load(f)
meanLeft = userMean["left"]
meanRight = userMean["right"]

In [5]:
eigenvaluesL, eigenvectorsL = np.linalg.eig(meanLeft)

eigenvaluesR, eigenvectorsR = np.linalg.eig(meanRight)

In [6]:
eigenvectorsLeft = torch.tensor(eigenvectorsL, dtype = torch.float32)
eigenvectorsRight = torch.tensor(eigenvectorsR, dtype = torch.float32)

In [7]:
def fullSessionPaths(dataset, root):
    sessions = [s["session"] for s in dataset]
    return [Path(root) / f"{session}.hdf5" for session in sessions]

In [8]:
print(config)

{'user': 'user0', 'dataset': {'train': [{'user': 43037958, 'session': '2020-12-17-1608244656-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-17-1608255062-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-17-1608257601-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-17-1608268481-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-18-1608304463-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-18-1608314177-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-18-1608311446-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2020-12-17-1608220409-keystrokes-dca-study@1-e041d7d9-a53b-40f3-aabc-e9714072ca46'}, {'user': 43037958, 'session': '2

In [9]:
trainTransform = Compose([
    ToTensor(fields=['emg_left', 'emg_right'], stack_dim = 1),
    ZNormalizeTime(),
    RandomBandRotation(),
    TemporalAlignmentJitter(max_offset = 120, stack_dim = 1),
    SlidingCovariance(eigenvectorsL = eigenvectorsLeft, eigenvectorsR = eigenvectorsRight, approxDiag = approxDiag)
])

valTransform = Compose([
    ToTensor(fields=['emg_left', 'emg_right'], stack_dim = 1),
    ZNormalizeTime(),
    SlidingCovariance(eigenvectorsL = eigenvectorsLeft, eigenvectorsR = eigenvectorsRight, approxDiag = approxDiag)
])

testTransform = Compose([
    ToTensor(fields=['emg_left', 'emg_right'], stack_dim = 1),
    ZNormalizeTime(),
    SlidingCovariance(eigenvectorsL = eigenvectorsLeft, eigenvectorsR = eigenvectorsRight, approxDiag = approxDiag)
])

trainSessions = fullSessionPaths(config.dataset.train, config.dataset.root)
valSessions = fullSessionPaths(config.dataset.val, config.dataset.root)
testSessions = fullSessionPaths(config.dataset.test, config.dataset.root)

In [10]:
datamodule = instantiate(
    config.datamodule,
    batch_size = config.batch_size,
    num_workers = config.num_workers,
    train_sessions = trainSessions,
    val_sessions = valSessions,
    test_sessions = testSessions,
    train_transform = trainTransform,
    val_transform = valTransform,
    test_transform = testTransform,
    _convert_ = "object"
)


datamodule.setup()

In [11]:
from emg2qwerty.lightning import TDSConvCTCModule
from emg2qwerty.charset import charset
import torch.optim as optim
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

In [12]:
model = TDSConvCTCModule(
    in_features = 16 * 16,
    mlp_features = config.module.mlp_features,
    block_channels = config.module.block_channels,
    kernel_width = config.module.kernel_width,
    optimizer = config.optimizer,
    lr_scheduler = config.lr_scheduler,
    decoder = config.decoder,
    share_hand_weights = False
)

checkpointCB = ModelCheckpoint(
    monitor = 'val/loss',
    save_top_k = 1,
    mode = 'min',
    filename = 'best-model',
    save_last = True,
    dirpath = './checkpoints'
)

In [13]:
lrMonitor = LearningRateMonitor(logging_interval = 'epoch')

trainer = pl.Trainer(
    accelerator = 'gpu',
    devices = 1,
    max_epochs = config.trainer.max_epochs,
    callbacks = [checkpointCB, lrMonitor],
    default_root_dir = './outputs',
    log_every_n_steps = 1
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
""" Train the model. Skip this cell if you simpy want simply test with the given checkpoints. """

trainer.fit(model, datamodule = datamodule)

In [14]:
if approxDiag:
    where = "withApproxDiag"
else: 
    where = "withoutApproxDiag"

checkpoint = torch.load("/mnt/dataDrive/emgFullCorpora/toUpload/DATA/emg2qwerty/" + where + "/" + user + ".ckpt")
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [15]:
trainer.validate(model, datamodule = datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Validation DataLoader 0:   5%|▌         | 1/20 [00:00<00:07,  2.56it/s]

/home/k2/miniconda3/envs/emg2qwerty/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Validation DataLoader 0: 100%|██████████| 20/20 [00:01<00:00, 13.44it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val/CER            18.492643356323242
         val/DER            2.8898062705993652
         val/IER             4.837514400482178
         val/SER             10.7653226852417
        val/loss            0.9733709096908569
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val/loss': 0.9733709096908569,
  'val/CER': 18.492643356323242,
  'val/IER': 4.837514400482178,
  'val/DER': 2.8898062705993652,
  'val/SER': 10.7653226852417}]

In [16]:
trainer.test(model, datamodule = datamodule)         

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/CER            22.591007232666016
        test/DER             2.002141237258911
        test/IER             6.723768711090088
        test/SER            13.865096092224121
        test/loss           1.1161961555480957
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/loss': 1.1161961555480957,
  'test/CER': 22.591007232666016,
  'test/IER': 6.723768711090088,
  'test/DER': 2.002141237258911,
  'test/SER': 13.865096092224121}]